In [159]:
import tensorflow 
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Reshape
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import pickle

In [90]:
corpus = """Today, all Indians in the country and also abroad are celebrating the festival of independence. On this day of sacred festival of independence, the prime servant of India extends greetings to all dear countrymen.

I am present amidst you not as the Prime Minister, but as the Prime Servant. The freedom struggle was fought for so many years, so many generations laid down their lives, innumerable people sacrificed their lives and youth, spent their entire lives behind bars. Today, I pay my respect, greetings and homage to all those who laid their lives for the country's independence.

I also pay my respects to the crores of citizens of this country on the pious occasion of India's independence, and recall all those martyrs who had laid down their lives in India's struggle for freedom. The day of independence is a festival when we take a solemn pledge of working for the welfare of mother India, and also for the welfare of the poor, oppressed, dalits, the exploited & the backward people of our country.

My dear countrymen, a national festival is an occasion to refine and rebuild the national character. This National festival inspires us to resolve ourselves to lead a life where our character gets refined further, to dedicate ourselves to the nation and our every activity is linked to the interest of the nation and only then this festival of freedom can be a festival of inspiration to take India to newer heights. My dear countrymen, this nation has neither been built by political leaders nor by rulers nor by governments. This nation has been built by our farmers, our workers, our mothers and sisters, our youth. The country has reached here today because of generation to generation rigours undertaken by our sages, our saints, our maestros, our teachers, our scientists and social workers. These great people and these great generations, who had worked for the country throughout their lives, deserve our deepest respect. This is the beauty of India's Constitution, this is its capability which has made it possible that today a boy from small town, a poor family has got the opportunity to pay homage to the tri-colour of India at the ramparts of 
Lal Quila (Red Fort). This is the strength of India's democracy. This is an invaluable legacy which we have inherited from our architects of the constitution. I pay my respects to those architects of the constitution of India today.

Brothers and sisters, today if we have reached here after independence, it is because of the contribution of all the Prime Ministers, all the governments and even the governments of all the States. I want to express my feelings of respect and gratitude to all those previous governments and ex-Prime Ministers who have endeavoured to take our present day India to such heights and who have added to the country's glory.

This country has been built on such foundation of ancient cultural heritage, where we were told of only one
mantra during 
Vedic period, which is indicative of our work culture, which we have learnt, we have memorized 
"""

In [91]:
tokenizer = Tokenizer()

In [93]:
tokenizer.fit_on_texts([corpus])

In [94]:
len(tokenizer.word_index)

212

In [95]:
input = []
for sentence in corpus.split('\n'):
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

    for i in range(1, len(tokenized_sentence)):
        input.append(tokenized_sentence[:i+1])




In [96]:
max_len = max([len(x) for x in input])

In [97]:
max_len

195

In [98]:
padded_seq = pad_sequences(input, maxlen=max_len, padding="pre")

In [99]:
padded_seq

array([[  0,   0,   0, ...,   0,  11,   8],
       [  0,   0,   0, ...,  11,   8,  84],
       [  0,   0,   0, ...,   8,  84,  46],
       ...,
       [  0,   0,   0, ...,  21, 211,  19],
       [  0,   0,   0, ..., 211,  19,  21],
       [  0,   0,   0, ...,  19,  21, 212]])

In [100]:
X = padded_seq[:,:-1]

In [101]:
Y = padded_seq[:,-1]

In [102]:
X.shape

(506, 194)

In [103]:
Y.shape

(506,)

In [104]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [149]:
model = Sequential()

model.add(Embedding(input_dim=213, output_dim=100, input_length=max_len-1))
model.add(LSTM(units=150))
model.add(Dense(units=213, activation="softmax"))

In [146]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 194, 100)          21300     
                                                                 
 lstm_25 (LSTM)              (None, 150)               150600    
                                                                 
 dense_10 (Dense)            (None, 213)               32163     
                                                                 
Total params: 204,063
Trainable params: 204,063
Non-trainable params: 0
_________________________________________________________________


In [150]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [151]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=70, batch_size=4)

Epoch 1/70


101/101 [==============================] - 36s 334ms/step - loss: 5.2382 - accuracy: 0.0470 - val_loss: 5.2576 - val_accuracy: 0.0980
Epoch 2/70
101/101 [==============================] - 34s 336ms/step - loss: 4.9032 - accuracy: 0.0569 - val_loss: 5.3879 - val_accuracy: 0.0980
Epoch 3/70
101/101 [==============================] - 33s 327ms/step - loss: 4.7692 - accuracy: 0.0569 - val_loss: 5.4205 - val_accuracy: 0.0980
Epoch 4/70
101/101 [==============================] - 33s 325ms/step - loss: 4.6078 - accuracy: 0.0842 - val_loss: 5.6067 - val_accuracy: 0.0784
Epoch 5/70
101/101 [==============================] - 33s 323ms/step - loss: 4.3324 - accuracy: 0.0990 - val_loss: 5.6049 - val_accuracy: 0.0882
Epoch 6/70
101/101 [==============================] - 32s 319ms/step - loss: 3.9084 - accuracy: 0.1485 - val_loss: 5.5969 - val_accuracy: 0.0980
Epoch 7/70
101/101 [==============================] - 41s 408ms/step - loss: 3.4247 - accuracy: 0.2129 - val_loss: 5.7860 - val_accuracy: 0.0

In [157]:
text = "india is the country of"
for i in range(300):
    token_text = tokenizer.texts_to_sequences([text])[0]
    padded = pad_sequences([token_text], maxlen=max_len-1, padding="pre")
    pos = np.argmax(model.predict(padded), axis=1)[0]
    for word, index in tokenizer.word_index.items():
        if index==pos:
            text = text+" "+word
print(text)            

1/1 [==============================] - 0s 32ms/step
india is the country of the prime minister but as the prime servant the freedom struggle was fought for so many generations laid down down innumerable innumerable innumerable their lives in india's struggle for the day of independence is a solemn when we take a solemn solemn working for for the welfare of mother india and also welfare the welfare of the poor oppressed the backward people of our only then of our many our many generations laid down their lives innumerable their lives for the country's independence and youth the country's independence is a solemn a beauty of working for the welfare of mother india and also the welfare of the poor oppressed the backward people of our even the backward people of our fought our many generations who had laid down innumerable innumerable innumerable their lives in india's struggle for the day of independence is a solemn when we take a solemn solemn working for for the welfare of mother india 

In [158]:
model.save("lstm_text_gen.h5")

In [160]:
pickle.dump(tokenizer, open("tokenizer.pkl", "wb"))